In [10]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle as pkl
import sys

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *

In [11]:
path = "C:/Users/gsamp/OneDrive/Documents/AI-3/2n Semestre/Projecte de Síntesi 2/GraphAnomaly/dades_guillem/"
df_classes = pd.read_csv(path + "elliptic_txs_classes.csv") # Nodes' labels
df_edges = pd.read_csv(path + "elliptic_txs_edgelist.csv") # Edges
df_features = pd.read_csv(path + "elliptic_txs_features.csv", header=None) # Nodes' features

# Change column names of df_features
colNames1 = {'0': 'txId', 1: "Time step"}
colNames2 = {str(ii+2): "Local_feature_" + str(ii+1) for ii in range(93)}
colNames3 = {str(ii+95): "Aggregate_feature_" + str(ii+1) for ii in range(72)}

colNames = dict(colNames1, **colNames2, **colNames3 )
colNames = {int(jj): item_kk for jj,item_kk in colNames.items()}

df_features = df_features.rename(columns=colNames)

In [12]:
# Specify the path to your pickle file
pickle_file_path = 'C:\\Users\\gsamp\\OneDrive\\Documents\\AI-3\\2n Semestre\\Projecte de Síntesi 2\\GraphAnomaly\\elipticData_graph.pkl'

# Open the pickle file in binary mode
with open(pickle_file_path, 'rb') as f:
    # Load the data from the pickle file
    G = pkl.load(f)


In [13]:
df_merged = pd.merge(left=df_features, right=df_classes, on="txId", how="left")
df_merged = df_merged.set_index('txId')

APPLY PCA

In [14]:
# Standardize the features (important for PCA)
scaler = StandardScaler()

df_pca = df_merged.drop(columns=['class'])
scaled_data = scaler.fit_transform(df_pca)

# Apply PCA
pca = PCA(n_components=75)  # You can choose the number of components you want to keep
principal_components = pca.fit_transform(scaled_data)

# Create a DataFrame for the principal components
columns = [f"PC{i+1}" for i in range(principal_components.shape[1])]
principal_df = pd.DataFrame(data=principal_components, columns=columns)


explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = explained_variance_ratio.sum()

print(f"Explained variance ratio: {cumulative_variance_ratio}")
print(f"Data reduction, from shape {df_pca.shape} to {principal_df.shape}")

#afegir dues columnes per poder aplicar logistic regression
principal_df['txId'] = df_pca.index
principal_df['class'] = list(df_merged['class'])

Explained variance ratio: 0.99178286404931
Data reduction, from shape (203769, 166) to (203769, 75)


Topological features

In [15]:
df_complete = pd.merge(df_features, df_classes, on='txId', how='left')
df_anotated = df_complete.loc[df_complete['class'].isin(['1', '2'])] 
df_edges = df_edges.loc[(df_edges.txId1.isin(df_anotated.txId)) & (df_edges.txId2.isin(df_anotated.txId))]
generate_graph = True
if generate_graph:
    # Create an empty graph
    G2 = nx.Graph()

    for _, row in df_anotated.iterrows():
        # Extract node ID and attributes
        node_id = row['txId']
        node_attributes = row.drop('txId').to_dict()
        
        # Add node to the graph with its attributes
        G2.add_node(node_id, **node_attributes)

    # Add edges to the graph
    for _, row in df_edges.iterrows():
        G2.add_edge(row['txId1'], row['txId2'])

In [16]:
# Additional topological features
degree_centrality = nx.degree_centrality(G2)
print(1)
betweenness_centrality = nx.betweenness_centrality(G2)
print(1)
clustering_coefficient = nx.clustering(G2)
print(1)
eigenvector_centrality = nx.eigenvector_centrality(G2)
print(1)
pagerank_centrality = nx.pagerank(G2)
print(1)
closeness_centrality = nx.closeness_centrality(G2)
print(1)
# avg_shortest_path_length = nx.average_shortest_path_length(G2)
print(1)
assortativity = nx.degree_assortativity_coefficient(G2)
print(1)
node_degrees = dict(G2.degree())



1
1
1
1
1
1
1
1


In [25]:
[degree_centrality[node], betweenness_centrality[node], clustering_coefficient[node]
                            #  eigenvector_centrality[node], pagerank_centrality[node], closeness_centrality[node],
                            #   assortativity[node]]

[0.0013315293258595881, 0.00021842204302680992, 0]

In [22]:
for n1, n2  in  zip(G2.nodes(), degree_centrality.keys()):
    if n1 == n2:
        print("aaa")
    break

aaa


In [23]:
feature_vectors = {}
for node in G2.nodes():
    feature_vectors[node] = [degree_centrality[node], betweenness_centrality[node], clustering_coefficient[node],
                             eigenvector_centrality[node], pagerank_centrality[node], closeness_centrality[node],
                              assortativity[node]]#, node_degrees[node]]

IndexError: invalid index to scalar variable.

In [ ]:
file_name = "topol_ft_dict.pkl"
with open(file_name, 'wb') as f:
    pkl.dump(feature_vectors, f)

Logistic regression

In [38]:
def lr(X, y, message: str):
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

    # Train the Logistic Regression model
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    
    with open('results.txt', 'a') as f:
        sys.stdout = f
        print(message)
        
        accuracy = accuracy_score(y_test, y_pred)
        print("Accuracy: {:.2f}%".format(accuracy * 100))

        precision = precision_score(y_test, y_pred)
        print(f"Precision: {precision * 100}%")

        # precision2 = precision_score(y_test, y_pred, pos_label='1')
        # print("Precision for illicit nodes: {:.2f}%".format(precision2 * 100))
        
        recall = recall_score(y_test, y_pred)
        print("Recall: {:.2f}%".format(recall * 100))
        
        f1score = f1_score(y_test, y_pred)
        print("F1 Score: {:.2f}%".format(f1score * 100))
        
        # Reset stdout back to the console
        sys.stdout = sys.__stdout__
        

In [40]:
X = principal_df.loc[principal_df['class'].isin(['1', '2'])].drop(columns=['txId', 'class'])
y = [1 if classe == '1' else 0 for classe in principal_df.loc[principal_df['class'].isin(['1', '2'])]['class']]

lr(X, y, message = "\nLogistic regression applied with PCA: \n")

X = df_merged.loc[df_merged['class'].isin(['1', '2'])].drop(columns=[1, 'class'])
y = [1 if classe == '1' else 0 for classe in df_merged.loc[df_merged['class'].isin(['1', '2'])]['class']]
lr(X, y, message = "\nLogistic regression applied to the raw data: \n")

c:\Users\gsamp\anaconda3\envs\visionEnv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
